## Project 1 Text Mining

##### Team: Elisa Scocco | Maria Simakova Mariukha | Victor Sobottka

Possible Festivals for research (I think we can also just go for the summer period)

https://www.locabarcelona.com/es/vivir-barcelona/fiestas/

### Event: Primavera Sound Barcelona

Cities:

- Barcelona
- Valencia

Valencia was chosen because have similarities with Barcelona in terms to be a coastal city, in the same country, but without being target in the same way that Barcelona for international tourists

Dates:

Prior_Event: 21/05/2025 - 25/05/2025

Event_Period: 04/06/2025 - 08/06/2025

In [1]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
import time
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import pandas as pd 

# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [2]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder) # you can predefine where you wanna store things in case its needed
    profile.set_preference("browser.download.folderList", 2) # 0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    profile.set_preference("browser.download.manager.showWhenStarting", False) #I dont wanna see a pop up for each download so i swith it off
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True) #dont want the pdf viewer to open

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [46]:
# lets open booking:

dfolder='./downloads'
geko_path = r"C:\Users\elisa\OneDrive\Desktop\Text mining\TA 1\geckodriver.exe"
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)


In [47]:
#Let's close the cookie
cookie='//div[@id="onetrust-button-group"]//button[@id="onetrust-reject-all-handler"]'


check_obscures(browser,cookie , type='xpath')

True

In [48]:
browser.find_element(by=By.XPATH,value='//*[@id=":rh:"]').click()

# in the booking page I inspected the element of where to go and look for the id


## Input the place: Send your desired destination and combine with the element above

The method `.send_keys()` allows you to send python inputs into the browse and interact with it. 

In [49]:
# Later the place will be changed to Valencia to search for hotels in the control city
place = 'Barcelona'
search1 = browser.find_element(by='xpath',value='//*[@id=":rh:"]')
# Wait briefly before submitting the input
time.sleep(1)

search1.send_keys(place)

# Wait for the search results to load
time.sleep(5) 

# We start building the dataframe where we will store all the information we need for the regression analysis.
df = pd.DataFrame([place], columns = ['City'])
print(df)

        City
0  Barcelona


## Now let's input the dates.

I am going to use a CSS selector. This part is one of the trickiest. booking structure does not allow to enter dates but you would need to click on them.

In [50]:

css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

browser.find_element('css selector',css).click()



## Now let's get all the dates. Complete the following path in order to get to the dates:


In [52]:
# XPath for the "Next" month button 
first_page_button_xpath = '//*[@id="calendar-searchboxdatepicker"]/div/div[1]/button'
next_button_xpath = '//*[@id="calendar-searchboxdatepicker"]/div/div[1]/button[2]'
path='//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa ef091eb985"]'


# Function to click "Next" until May is found
def navigate_to_may():
    while True:
        
        # Retrieve all the dates in the current view
        dates = browser.find_elements('xpath', path )
        
        
        # Check if any of the dates is in May
        for date in dates:
            data_date = date.get_attribute("data-date")
            if "2025-05" in data_date:
                print("Found May!")
                return  # We found May, exit the function
        
        # If May is not found, click the "Next" button to go to the next month
        
        i = [first_page_button_xpath,next_button_xpath,next_button_xpath,next_button_xpath]
        for j in i:
            next_button = browser.find_element('xpath', j)
            next_button.click()
            time.sleep(1)   


navigate_to_may()

# Retrieve dates for May
dates = browser.find_elements('xpath', path)


Found May!
2025-05-01
2025-05-02
2025-05-03
2025-05-04
2025-05-05
2025-05-06
2025-05-07
2025-05-08
2025-05-09
2025-05-10
2025-05-11
2025-05-12
2025-05-13
2025-05-14
2025-05-15
2025-05-16
2025-05-17
2025-05-18
2025-05-19
2025-05-20
2025-05-21
2025-05-22
2025-05-23
2025-05-24
2025-05-25
2025-05-26
2025-05-27
2025-05-28
2025-05-29
2025-05-30
2025-05-31


In [53]:
#We now store the dates into a column, and later we will add it to our dataframe
date_list_may = []

from_day = "21"  
to_day = "25"  

for date in dates:
    if date.get_attribute("data-date") == f"2025-05-{from_day}":
        date.click()
        date_list_may.append(f"2025-05-{from_day}")
        time.sleep(2)
    if date.get_attribute("data-date") == f"2025-05-{to_day}":
        date.click()
        date_list_may.append(f"2025-05-{to_day}")
        time.sleep(2)
        break




In [54]:
date_list_may

['2025-05-21', '2025-05-25']

In [55]:
#Function string_to_date_period to serve as input in the df
import pandas as pd

def string_to_date_period(date_list):
    # Convert the first and last dates in the list to datetime objects
    start_date = pd.to_datetime(date_list[0])
    end_date = pd.to_datetime(date_list[-1])
    
    # Format the start and end dates as strings
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    # Create the date period string
    date_period = f"{start_date_str} : {end_date_str}"
    return date_period


# Call the function and get the date period
date_period = string_to_date_period(date_list_may)

# Place the date period in the first row under "Date Period"
df.loc[0, 'Date Period'] = date_period

# Drop rows where both "City" and "Date Period" are NaN
df.dropna(how='all', inplace=True)

# Print the updated DataFrame
print(df)


        City              Date Period
0  Barcelona  2025-05-21 : 2025-05-25


In [56]:
# Let's click on search to find all the hotels for the city selected for the time selected
my_xpath='//div[@class="e22b782521 d12ff5f5bf"]//button[@class="a83ed08757 c21c56c305 a4c1805887 f671049264 a2abacf76b c082d89982 cceeb8986b b9fd3c6b3c"]'



check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

Message: The element with the reference 0434f1e2-a11f-4a63-9263-7f47f80e5fe6 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:725:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:405:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:263:20
cloneObject@chrome://remote/content/marionette/json.sys.mjs:59:24
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:293:16
json.deserialize@chrome://remote/content/marionette/json.sys.mjs:297:10
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCom

In [57]:
# Let's obscure the pop up asking to sign in
popup='//div[@class="abcc616ec7 cc1b961f14 c180176d40 f11eccb5e8 ff74db973c"]'


check_obscures(browser,popup , type='xpath')

time.sleep(1)

## Code to Scroll down the page

In [58]:
#Code to scroll down the page  and load more hotels
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

current_scroll_position = 0  # Initialize the scroll position

while True:
    try:
        # Try to find the "Cargar más resultados" button
        load_more_button = browser.find_element('xpath', '//button[span[text()="Cargar más resultados"]]')

        # If the button is found, scroll it into view and click
        browser.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Cargar más resultados' button")

        # Wait for new content to load
        time.sleep(2)

    except NoSuchElementException:
        # If button not found, scroll down incrementally to reach the bottom of the page
        print("Scrolling down to load more results...")

        # Scroll incrementally until the bottom of the page is reached
        scroll_height = browser.execute_script("return document.body.scrollHeight;")  # Total page height

        while current_scroll_position < scroll_height:
            browser.execute_script(f"window.scrollTo({current_scroll_position}, {current_scroll_position + 11500});")
            try:
                current_scroll_position += 1100  # Increment scroll position
                time.sleep(1)
            except:
                current_scroll_position += 1100  # Increment scroll position
                time.sleep(1)

            # Check the updated scroll height after each increment
            scroll_height = browser.execute_script("return document.body.scrollHeight;")
        
        print("Reached the bottom of the page. Stopping...")
        break  # Exit the outer loop after reaching the bottom

    except ElementClickInterceptedException:
        # Handle cases where the button is not clickable
        print("Button is not clickable. Retrying...")
        browser.execute_script("window.scrollBy(0, 200);")  # Scroll slightly down
        time.sleep(1)


Scrolling down to load more results...
Reached the bottom of the page. Stopping...


In [59]:
# The previous code might scroll down to the bottom and it might not find the "Cargar mas button". We scroll up a litlle bit to ensure this.
current_scroll_position = browser.execute_script("return window.pageYOffset;")  # Get current scroll position

while True:
    try:
        # Scroll up slightly to make the button visible
        browser.execute_script("window.scrollBy(0, -200);")  # Scroll up a bit
        time.sleep(1)  # Wait for the page to adjust

        # Try to find the "Cargar más resultados" button
        load_more_button = browser.find_element('xpath', '//button[span[text()="Cargar más resultados"]]')
        
        # If the button is found, click it
        load_more_button.click()
        print("Clicked 'Cargar más resultados' button")

        # Wait for new content to load
        time.sleep(2)
    
    except NoSuchElementException:
        print("Button not found. Stopping...")
        break  # Exit loop if the button is no longer found

    except ElementClickInterceptedException:
        print("Button is not clickable. Adjusting...")
        browser.execute_script("window.scrollBy(0, -50);")  # Scroll up slightly more
        time.sleep(1)


Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resul

In [ ]:

hotels = browser.find_elements('xpath', '//div[@data-testid="title"]')


1. Be Mate Paseo de Gracia
2. Hotel La Pau 4 Sup
3. Four Points by Sheraton Barcelona Airport
4. Tembo Barcelona
5. Axel TWO Barcelona 4 Sup - Adults Only
6. Casa Consell Gran Via, Guest House
7. Good-Home Paseo de Gracia
8. Ohla Eixample
9. Hotel Paxton Barcelona
10. Ohla Barcelona
11. Sixtytwo Hotel
12. Charmsuites Paralel
13. Charmsuites Nou Rambla
14. Yeah Barcelona Hostel
15. Nicebarcelonaapartments
16. Hostel Friends Barcelona
17. ApartEasy - Gracia Apartments
18. ibis Styles Barcelona City Bogatell
19. Hostal Centric
20. Hostal Radio Barcelona
21. Lovely Appart & Sky View SAGRADA FAMILIA
22. Casa Lit Barcelona
23. Hotel Via Augusta
24. B Hotel
25. Royal Ramblas
26. Hesperia Barcelona Barri Gòtic
27. Holiday Inn Express Barcelona City 22@, an IHG Hotel
28. Hotel Viladomat
29. Hostel New York
30. Hostal Lleida
31. Golden Hotel Barcelona
32. Serennia Apartamentos Ramblas - Plaça Catalunya
33. MH Apartments Gracia
34. 08028 Apartments
35. Barcelona Hotel Colonial
36. TSA Gran Via
37

In [67]:
hotels_list=[]
for i in hotels:
    print(i.text)
    hotels_list.append(i.text)
len(hotels_list)

Be Mate Paseo de Gracia
Hotel La Pau 4 Sup
Four Points by Sheraton Barcelona Airport
Tembo Barcelona
Axel TWO Barcelona 4 Sup - Adults Only
Casa Consell Gran Via, Guest House
Good-Home Paseo de Gracia
Ohla Eixample
Hotel Paxton Barcelona
Ohla Barcelona
Sixtytwo Hotel
Charmsuites Paralel
Charmsuites Nou Rambla
Yeah Barcelona Hostel
Nicebarcelonaapartments
Hostel Friends Barcelona
ApartEasy - Gracia Apartments
ibis Styles Barcelona City Bogatell
Hostal Centric
Hostal Radio Barcelona
Lovely Appart & Sky View SAGRADA FAMILIA
Casa Lit Barcelona
Hotel Via Augusta
B Hotel
Royal Ramblas
Hesperia Barcelona Barri Gòtic
Holiday Inn Express Barcelona City 22@, an IHG Hotel
Hotel Viladomat
Hostel New York
Hostal Lleida
Golden Hotel Barcelona
Serennia Apartamentos Ramblas - Plaça Catalunya
MH Apartments Gracia
08028 Apartments
Barcelona Hotel Colonial
TSA Gran Via
Capri by Fraser Barcelona
Hotel & Spa Villa Olimpica Suites
Unite Hostel Barcelona
Mercer House Bòria BCN
Hotel California
Hotel Sant Agu

960

In [ ]:
#We strore all the hotel info we found into the dataframe, together with the price
df = df.reindex(df.index.repeat(len(hotels_list)))

# Reset index to ensure a clean, sequential index
df.reset_index(drop=True, inplace=True)

# Now you can add the 'Hotel' and 'Rating' columns to the original DataFrame if it is repeated
df['Hotel'] = hotels_list


prices = browser.find_elements(By.XPATH, '//span[@data-testid="price-and-discounted-price"]')
for index, price_element in enumerate(prices, start=1):
    price = price_element.text  
    print(f"{index}. {price}")

prices_list = []
for price_element in prices:
    price_text = price_element.text 
    prices_list.append(price_text) 

numeric_prices = []

for price in prices_list:
    numeric_price = float(price.replace('€', '').replace('.', '').strip())
    numeric_prices.append(numeric_price)
    

df['Prices'] = numeric_prices


1. € 1.443
2. € 2.568
3. € 531
4. € 1.707
5. € 976
6. € 1.057
7. € 815
8. € 2.111
9. € 1.620
10. € 2.791
11. € 2.819
12. € 895
13. € 895
14. € 874
15. € 892
16. € 573
17. € 865
18. € 941
19. € 833
20. € 498
21. € 961
22. € 1.113
23. € 944
24. € 1.338
25. € 1.298
26. € 1.300
27. € 962
28. € 1.236
29. € 314
30. € 772
31. € 1.036
32. € 1.547
33. € 1.121
34. € 762
35. € 1.195
36. € 1.450
37. € 1.087
38. € 1.043
39. € 1.118
40. € 1.400
41. € 1.358
42. € 1.090
43. € 1.699
44. € 1.852
45. € 1.417
46. € 1.594
47. € 1.132
48. € 1.594
49. € 941
50. € 913
51. € 1.244
52. € 978
53. € 1.095
54. € 1.180
55. € 1.634
56. € 3.061
57. € 3.201
58. € 2.329
59. € 2.527
60. € 676
61. € 6.211
62. € 1.116
63. € 1.424
64. € 921
65. € 1.092
66. € 1.361
67. € 1.217
68. € 1.181
69. € 1.495
70. € 831
71. € 1.374
72. € 1.300
73. € 1.650
74. € 1.033
75. € 1.135
76. € 1.245
77. € 1.073
78. € 1.407
79. € 1.316
80. € 1.802
81. € 1.130
82. € 1.300
83. € 1.333
84. € 699
85. € 1.200
86. € 1.113
87. € 1.019
88. € 731
89. €

In [69]:
df

,City,Date Period,Hotel,Prices
0,Barcelona,2025-05-21 : 2025-05-25,Be Mate Paseo de Gracia,1443.0
1,Barcelona,2025-05-21 : 2025-05-25,Hotel La Pau 4 Sup,2568.0
2,Barcelona,2025-05-21 : 2025-05-25,Four Points by Sheraton Barcelona Airport,531.0
3,Barcelona,2025-05-21 : 2025-05-25,Tembo Barcelona,1707.0
4,Barcelona,2025-05-21 : 2025-05-25,Axel TWO Barcelona 4 Sup - Adults Only,976.0
...,...,...,...,...
955,Barcelona,2025-05-21 : 2025-05-25,Hotel Coronado,720.0
956,Barcelona,2025-05-21 : 2025-05-25,Alojamiento Villarro,444.0
957,Barcelona,2025-05-21 : 2025-05-25,My Space Emirates - Fully equipped apartments ...,1312.0
958,Barcelona,2025-05-21 : 2025-05-25,Perfectly located 4bed 4bath apartment in Barc...,2376.0


In [72]:
from bs4 import BeautifulSoup

# Encuentra los enlaces de cada hotel en la página de resultados
hotel_links = []
hotels = browser.find_elements(By.CSS_SELECTOR, "a[data-testid='title-link']")  # Selector CSS para enlaces de hoteles
for hotel in hotels:
    hotel_links.append(hotel.get_attribute("href"))  # Guarda cada enlace completo


In [74]:
#We now eneter into each hotel link to obtain description and ratings info
descriptions = []
ratings = []

for hotel_url in hotel_links:
    # Abre el enlace del hotel
    browser.get(hotel_url)
    time.sleep(3)  # Espera a que la página del hotel cargue completamente

    # Usar BeautifulSoup para analizar la página del hotel
    soup = BeautifulSoup(browser.page_source, "lxml")
    
    # Buscar la descripción completa del hotel
    description = soup.find("p", class_="a53cbfa6de b3efd73f69")  
    if description:
        descriptions.append(description.text.strip())
    else:
        descriptions.append("Descripción no encontrada")

    browser.back()
    time.sleep(3)

# Extract the rating
    rating_container = soup.find("div", class_="a3b8729ab1 d86cee9b25")
    if rating_container:
        rating_text = rating_container.find("div", class_="ac4a7896c7")
        if rating_text:
            rating = rating_text.text.strip().replace(',', '.')  # Standardize decimal format
            rating = rating.split()[-1]  # Extract only the numeric rating

        else:
            rating = None  # No rating found
    else:
        rating = None  # No rating container found

    ratings.append(rating)

    # Go back to the results page
    browser.back()
    time.sleep(3)


In [76]:
df['Ratings'] = ratings
df['Description'] = descriptions

In [78]:
df.to_csv("dfhotelsbcnwithdescrmay.csv", index=False)  


## Let's check for June now

We follow the exact same steps, but now we look for June's hotel information

In [19]:
# lets open booking:

dfolder='./downloads'
geko_path = r"C:\Users\elisa\OneDrive\Desktop\Text mining\TA 1\geckodriver.exe"
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

cookie='//div[@id="onetrust-button-group"]//button[@id="onetrust-reject-all-handler"]'


check_obscures(browser,cookie , type='xpath')
time.sleep(2)  

browser.find_element(by=By.XPATH,value='//*[@id=":rh:"]').click()
time.sleep(1)
place = 'Barcelona'
search1 = browser.find_element(by='xpath',value='//*[@id=":rh:"]')
# Wait briefly before submitting the input
time.sleep(1)

search1.send_keys(place)

# Wait for the search results to load
time.sleep(5)  


df2 = pd.DataFrame([place], columns = ['City'])
print(df2)

css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

browser.find_element('css selector',css).click()

        City
0  Barcelona


In [20]:
# XPath for the "Next" month button (you may need to adjust this depending on the website structure)
first_page_button_xpath = '//*[@id="calendar-searchboxdatepicker"]/div/div[1]/button'
next_button_xpath = '//*[@id="calendar-searchboxdatepicker"]/div/div[1]/button[2]'
path='//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa ef091eb985"]'


# Function to click "Next" until June is found
def navigate_to_june():
    while True:
        
        # Retrieve all the dates in the current view
        dates = browser.find_elements('xpath', path )
        
        
        # Check if any of the dates is in June
        for date in dates:
            data_date = date.get_attribute("data-date")
            if "2025-06" in data_date:
                print("Found June!")
                return  # We found June, exit the function
        
        # If June is not found, click the "Next" button to go to the next month
        
        i = [first_page_button_xpath,next_button_xpath,next_button_xpath,next_button_xpath,next_button_xpath]
        for j in i:
            next_button = browser.find_element('xpath', j)
            next_button.click()
            time.sleep(1)  # Wait for the calendar to update   


# Call the function to navigate to June
navigate_to_june()

# Retrieve and print dates for June
dates = browser.find_elements('xpath', path)


Found June!
2025-06-01
2025-06-02
2025-06-03
2025-06-04
2025-06-05
2025-06-06
2025-06-07
2025-06-08
2025-06-09
2025-06-10
2025-06-11
2025-06-12
2025-06-13
2025-06-14
2025-06-15
2025-06-16
2025-06-17
2025-06-18
2025-06-19
2025-06-20
2025-06-21
2025-06-22
2025-06-23
2025-06-24
2025-06-25
2025-06-26
2025-06-27
2025-06-28
2025-06-29
2025-06-30


In [21]:
date_list_june = []

from_day = "04"  
to_day = "08"  

for date in dates:
    if date.get_attribute("data-date") == f"2025-06-{from_day}":
        date.click()
        date_list_june.append(f"2025-06-{from_day}")
        time.sleep(2)
    if date.get_attribute("data-date") == f"2025-06-{to_day}":
        date.click()
        date_list_june.append(f"2025-06-{to_day}")
        time.sleep(2)
        break



date_list_june


['2025-06-04', '2025-06-08']

In [23]:
#Function string_to_date_period to serve as input in the df
import pandas as pd

def string_to_date_period(date_list):
    # Convert the first and last dates in the list to datetime objects
    start_date = pd.to_datetime(date_list[0])
    end_date = pd.to_datetime(date_list[-1])
    
    # Format the start and end dates as strings
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    # Create the date period string
    date_period2 = f"{start_date_str} : {end_date_str}"
    return date_period2


# Call the function and get the date period
date_period2 = string_to_date_period(date_list_june)

# Place the date period in the first row under "Date Period"
df2.loc[0, 'Date Period'] = date_period2

# Drop rows where both "City" and "Date Period" are NaN
df2.dropna(how='all', inplace=True)

# Print the updated DataFrame
print(df2)


        City              Date Period
0  Barcelona  2025-06-04 : 2025-06-08


In [24]:
my_xpath='//div[@class="e22b782521 d12ff5f5bf"]//button[@class="a83ed08757 c21c56c305 a4c1805887 f671049264 a2abacf76b c082d89982 cceeb8986b b9fd3c6b3c"]'



check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')


popup='//div[@class="abcc616ec7 cc1b961f14 c180176d40 f11eccb5e8 ff74db973c"]'


check_obscures(browser,popup , type='xpath')

time.sleep(1)

In [26]:
#Code to scroll down the page
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

current_scroll_position = 0  # Initialize the scroll position

while True:
    try:
        # Try to find the "Cargar más resultados" button
        load_more_button = browser.find_element('xpath', '//button[span[text()="Cargar más resultados"]]')

        # If the button is found, scroll it into view and click
        browser.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Cargar más resultados' button")

        # Wait for new content to load
        time.sleep(2)

    except NoSuchElementException:
        # If button not found, scroll down incrementally to reach the bottom of the page
        print("Scrolling down to load more results...")

        # Scroll incrementally until the bottom of the page is reached
        scroll_height = browser.execute_script("return document.body.scrollHeight;")  # Total page height

        while current_scroll_position < scroll_height:
            browser.execute_script(f"window.scrollTo({current_scroll_position}, {current_scroll_position + 11500});")
            try:
                current_scroll_position += 500  # Increment scroll position
                time.sleep(1)
            except:
                current_scroll_position += 500  # Increment scroll position
                time.sleep(1)

            # Check the updated scroll height after each increment
            scroll_height = browser.execute_script("return document.body.scrollHeight;")
        
        print("Reached the bottom of the page. Stopping...")
        break  # Exit the outer loop after reaching the bottom

    except ElementClickInterceptedException:
        # Handle cases where the button is not clickable
        print("Button is not clickable. Retrying...")
        browser.execute_script("window.scrollBy(0, 200);")  # Scroll slightly down
        time.sleep(1)


Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resultados' button
Clicked 'Cargar más resul

In [27]:
current_scroll_position = browser.execute_script("return window.pageYOffset;")  # Get current scroll position

while True:
    try:
        # Scroll up slightly to make the button visible
        browser.execute_script("window.scrollBy(0, -200);")  # Scroll up a bit
        time.sleep(1)  # Wait for the page to adjust

        # Try to find the "Cargar más resultados" button
        load_more_button = browser.find_element('xpath', '//button[span[text()="Cargar más resultados"]]')
        
        # If the button is found, click it
        load_more_button.click()
        print("Clicked 'Cargar más resultados' button")

        # Wait for new content to load
        time.sleep(2)
    
    except NoSuchElementException:
        print("Button not found. Stopping...")
        break  # Exit loop if the button is no longer found

    except ElementClickInterceptedException:
        print("Button is not clickable. Adjusting...")
        browser.execute_script("window.scrollBy(0, -50);")  # Scroll up slightly more
        time.sleep(1)


Button not found. Stopping...


## Now we are almost ready

Lets try to get hotel names 

In [28]:
hotels = browser.find_elements('xpath', '//div[@data-testid="title"]')
hotels_list=[]
for i in hotels:
    print(i.text)
    hotels_list.append(i.text)
len(hotels_list)


df2 = df2.reindex(df2.index.repeat(len(hotels_list)))

# Reset index to ensure a clean, sequential index
df2.reset_index(drop=True, inplace=True)

# Now you can add the 'Hotel' columns to the original DataFrame if it is repeated
df2['Hotel'] = hotels_list


prices = browser.find_elements(By.XPATH, '//span[@data-testid="price-and-discounted-price"]')
for index, price_element in enumerate(prices, start=1):
    price = price_element.text  
    

prices_list = []
for price_element in prices:
    price_text = price_element.text 
    prices_list.append(price_text) 

numeric_prices = []

for price in prices_list:
    numeric_price = float(price.replace('€', '').replace('.', '').strip())
    numeric_prices.append(numeric_price)
    

df2['Prices'] = numeric_prices


Click&Flat Stylish Torrijos
Hotel La Pau 4 Sup
Be Mate Paseo de Gracia
Four Points by Sheraton Barcelona Airport
Tembo Barcelona
Axel TWO Barcelona 4 Sup - Adults Only
Ohla Eixample
Casa Consell Gran Via, Guest House
Ohla Barcelona
Sixtytwo Hotel
Hotel Paxton Barcelona
Hotel Viladomat
Casa Lit Barcelona
Hotel Rialto
Hotel Via Augusta
Chic & Basic Velvet
Catalonia Diagonal Centro
B Hotel
Catalonia Barcelona 505
Royal Ramblas
Catalonia Sagrada Familia
BCN Urbaness Hotels Del Comte
Catalonia Eixample 1864
Hesperia Barcelona Barri Gòtic
Hotel Concordia Barcelona
Catalonia Atenas
Hostal Central Barcelona
Gaudi Hotel
Catalonia Barcelona Plaza
Hotel Sant Agustí
Barcelona Hotel Colonial
Occidental Barcelona 1929
Hotel Gótico
Catalonia Passeig de Gràcia 4* Sup
Catalonia Portal de l'Angel
Aparthotel Bcn Montjuic
Catalonia Rigoletto
Catalonia Avinyo
Zenit Barcelona
Hotel California
Catalonia La Boquería
Hotel & Spa Villa Olimpica Suites
Ofelias Hotel 4Sup by Bondia Hotel Group
Mercer House Bòria 

In [29]:
df2

,City,Date Period,Hotel,Prices
0,Barcelona,2025-06-04 : 2025-06-08,Click&Flat Stylish Torrijos,2444.0
1,Barcelona,2025-06-04 : 2025-06-08,Hotel La Pau 4 Sup,2133.0
2,Barcelona,2025-06-04 : 2025-06-08,Be Mate Paseo de Gracia,1246.0
3,Barcelona,2025-06-04 : 2025-06-08,Four Points by Sheraton Barcelona Airport,508.0
4,Barcelona,2025-06-04 : 2025-06-08,Tembo Barcelona,2411.0
...,...,...,...,...
992,Barcelona,2025-06-04 : 2025-06-08,AB Sagrada Família Comfort,2319.0
993,Barcelona,2025-06-04 : 2025-06-08,Canela Homes BARCELONA FLORIDABLANCA,2357.0
994,Barcelona,2025-06-04 : 2025-06-08,Alcam Anglesola,2420.0
995,Barcelona,2025-06-04 : 2025-06-08,Stay U-nique Apartments GranDeGracia,2045.0


In [30]:
from bs4 import BeautifulSoup

# Encuentra los enlaces de cada hotel en la página de resultados
hotel_links = []
hotels = browser.find_elements(By.CSS_SELECTOR, "a[data-testid='title-link']")  # Selector CSS para enlaces de hoteles
for hotel in hotels:
    hotel_links.append(hotel.get_attribute("href"))  # Guarda cada enlace completo



In [31]:

# Imprime los enlaces de los hoteles encontrados
print(f"Enlaces de hoteles encontrados: {len(hotel_links)}")
for link in hotel_links:
    print(link)

# Extraer descripciones completas de cada hotel
descriptions = []
ratings = []

for hotel_url in hotel_links:
    # Abre el enlace del hotel
    browser.get(hotel_url)
    time.sleep(3)  # Espera a que la página del hotel cargue completamente

    # Usar BeautifulSoup para analizar la página del hotel
    soup = BeautifulSoup(browser.page_source, "lxml")
    
    # Buscar la descripción completa del hotel
    description = soup.find("p", class_="a53cbfa6de b3efd73f69")  # Ajusta esta clase según el HTML real de Booking
    if description:
        descriptions.append(description.text.strip())
    else:
        descriptions.append("Descripción no encontrada")

    # Opcional: Volver a la página de resultados
    browser.back()
    time.sleep(3)

# Extract the rating
    rating_container = soup.find("div", class_="a3b8729ab1 d86cee9b25")
    if rating_container:
        rating_text = rating_container.find("div", class_="ac4a7896c7")
        if rating_text:
            rating = rating_text.text.strip().replace(',', '.')  # Standardize decimal format
            rating = rating.split()[-1]  # Extract only the numeric rating

        else:
            rating = None  # No rating found
    else:
        rating = None  # No rating container found

    ratings.append(rating)

    # Go back to the results page
    browser.back()
    time.sleep(3)


Enlaces de hoteles encontrados: 997
https://www.booking.com/hotel/es/click-amp-flat-stylish-torrijos.es.html?aid=304142&label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4ARfIAQ_YAQHoAQH4AQKIAgGoAgO4AvHA7rwGwAIB0gIkN2JjM2VlNjktZWNkYy00YmViLTlkNTgtNzhlNTU2NzAwNTdh2AIF4AIB&ucfs=1&arphpl=1&checkin=2025-06-04&checkout=2025-06-08&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=2b51701c48230906&srepoch=1738252809&all_sr_blocks=925643901_364364526_2_0_0&highlighted_blocks=925643901_364364526_2_0_0&matching_block_id=925643901_364364526_2_0_0&sr_pri_blocks=925643901_364364526_2_0_0__244372&from=searchresults
https://www.booking.com/hotel/es/hostal-nilo.es.html?aid=304142&label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4ARfIAQ_YAQHoAQH4AQKIAgGoAgO4AvHA7rwGwAIB0gIkN2JjM2VlNjktZWNkYy00YmViLTlkNTgtNzhlNTU2NzAwNTdh2AIF4AIB&ucfs=1&arphpl=1&checkin=2025-06-04&checkout=2025-06-08&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=

In [35]:
df2.to_csv("dfhotelsbcnwithdescrjune.csv", index=False)  


We will now use the 4 csv file to complete part 2 and 3 of the Homework